# Deep Learning Tutorial 2: Training

In [4]:
from keras.models import Model
from keras.layers import *
from keras.utils.visualize_util import plot

import numpy as np

from callbacks import AUCHistory

Using Theano backend.


### Load our data

Data has been saved in a file named *data.npz*. It can be loaded using *np.load* and then behaves similar to a Python dictionary, where they keys *records* and *labels* corresponds to a numpy array for our input data and our labels, respectively.

In [27]:
saved = np.load("data.npz")
data = saved["records"]
labels = saved["labels"]

The input data is a 3-dimensional array, where the first dimension corresponds to the number of instances, the second dimension to the number of time steps, and the third dimension to the number of features.

More specifically, 74574 is the number of drives in our data, 70 the number of records per drive, and 78 features per record.

In [28]:
data.shape

(74574, 70, 15)

In [18]:
n_drives = data.shape[0]
n_records = data.shape[1]
n_features = data.shape[2]

In [19]:
input = Input(shape=(n_records, n_features), name="inputs")

In [20]:
x = input

In [21]:
x = Masking()(input)

In [22]:
x = LSTM(30, return_sequences=True)(x)
x = LSTM(30)(x)


In [23]:
output = Dense(1, activation='sigmoid', name='output')(x)

In [24]:
model = Model(input=input, output=output)

In [25]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy")

In [26]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
inputs (InputLayer)              (None, 70, 78)        0                                            
____________________________________________________________________________________________________
masking_2 (Masking)              (None, 70, 78)        0           inputs[0][0]                     
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 70, 20)        7920        masking_2[0][0]                  
____________________________________________________________________________________________________
lstm_4 (LSTM)                    (None, 20)            3280        lstm_3[0][0]                     
___________________________________________________________________________________________

In [4]:
model.fit(data, labels, nb_epoch=10, batch_size=50, validation_split=0.2, callbacks=[AUCHistory()])

NameError: name 'model' is not defined